In [1]:
from os import listdir
from os.path import isfile, join
import json
import pandas as pd
import numpy as np

In [2]:
from collections import Counter


In [3]:
import sys
sys.path.append("../")
import aux
import aux.metrics
import torch

In [4]:
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

In [5]:
dir = "flows_vi_experiments_results_1"

In [6]:
files = sorted([f for f in listdir(dir) if isfile(join(dir, f)) and f.endswith("_T=trained-nn.csv")])

file2samples = {}
for file in files:
    file2samples[file] = pd.read_csv(join(dir, file)).values[:, 1:]
    
print("\n".join(file2samples.keys()))

F=-1-0_S=100_E=4000-100_B=0.1-False-200_R=0_T=trained-nn.csv
F=-1-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.csv
F=-1-0_S=100_E=4000-100_B=1.0-False-200_R=0_T=trained-nn.csv
F=-1-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=trained-nn.csv
F=-1-0_S=10_E=4000-100_B=1.0-False-200_R=0_T=trained-nn.csv
F=0-0_S=100_E=4000-100_B=0.1-False-200_R=0_T=trained-nn.csv
F=0-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.csv
F=0-0_S=100_E=4000-100_B=1.0-False-200_R=0_T=trained-nn.csv
F=0-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=trained-nn.csv
F=0-0_S=10_E=4000-100_B=1.0-False-200_R=0_T=trained-nn.csv
F=11-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.csv
F=11-0_S=100_E=4000-100_B=1.0-False-200_R=0_T=trained-nn.csv
F=2-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.csv
F=2-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=trained-nn.csv
F=21-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.csv
F=21-0_S=100_E=4000-100_B=1.0-False-200_R=0_T=trained-nn.csv
F=21-1_S=100_E=4000-100_B=1.0-False-

In [7]:
baseline_samples = pd.read_csv(join(dir, "F=0-0_S=100_E=4000-100_B=0.1-False-200_R=0_T=bnn:-100-mcmc-posterior-samples.csv")).values[:, 1:]

In [8]:
def file2cfg(file):
    mapping = {
        "F": "flow / init",
        "S": "n_samples",
        "E": "max_epochs",
        "B": "Beta / Anneal / Iters",
        "R": "seed",
    }
    cfg = {}
    file = ".".join(file.split(".")[:-1])
    for p in file.split("_"):        
        k, v = p.split("=")
        k = mapping.get(k,k)
        cfg[k] = v.replace("-", " / ")
        # print(p, "=>", k, v)
    return cfg
    

In [9]:
def zoom_center(samples, window=20):
    if window is None:
        return samples
    
    w = window // 2
    n_samples = samples.shape[0]
    grid = int(np.sqrt(samples.shape[-1]))
    assert grid*grid == samples.shape[-1]
    c = grid//2
    samples = samples.reshape((-1, grid,grid))    
    samples = samples[:, (c-w):(c+w), (c-w):(c+w)]
    return samples.reshape((n_samples, -1))

In [10]:
def evaluate(zoom_window = None):
    results = []
    for file, samples in file2samples.items():
        # m = {"id": file}
        m = file2cfg(file)
        s1 = zoom_center(samples, zoom_window)
        s2 = zoom_center(baseline_samples, zoom_window)
        m.update(aux.metrics.compute_distribution_distances(torch.tensor(s1), torch.tensor(s2)))    
        m["file"] = file
        
        results.append(m)
        
    return pd.DataFrame(results).drop(columns=["T", "seed", "max_epochs"]).sort_values("2-Wasserstein")    

In [11]:
selected = []
zoom_windows = [None, 50, 20]
for zoom_window in zoom_windows:
    d = evaluate(zoom_window).head(10)
    display(d)
    
    selected.extend(list(d["file"].values))

,flow / init,n_samples,Beta / Anneal / Iters,1-Wasserstein,2-Wasserstein,Linear_MMD,Poly_MMD,RBF_MMD,Mean_MSE,Mean_L2,Mean_L1,Median_MSE,Median_L2,Median_L1,file
13,2 / 0,100,1.0 / True / 200,28.743283,29.041977,124.670739,8.755487e+05,0.102908,0.011192,0.105794,0.083986,0.017402,0.131915,0.101825,F=2-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=tr...
7,0 / 0,100,1.0 / False / 200,30.318780,30.587460,184.205663,1.325959e+06,0.106888,0.019225,0.138654,0.107732,0.033638,0.183406,0.133261,F=0-0_S=100_E=4000-100_B=1.0-False-200_R=0_T=t...
3,/ 1 / 0,100,1.0 / True / 200,30.537219,30.910947,248.505668,1.968807e+06,0.113019,0.021488,0.146589,0.114562,0.037451,0.193523,0.138481,F=-1-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=t...
19,31 / 0,10,0.3 / False / 200,30.756670,31.212547,236.748824,1.870807e+06,0.131441,0.024140,0.155371,0.118722,0.041009,0.202508,0.142763,F=31-0_S=10_E=4000-100_B=0.3-False-200_R=0_T=t...
17,3 / 0,10,0.3 / False / 200,31.012717,31.384079,282.528581,2.264353e+06,0.136218,0.025434,0.159479,0.122904,0.038571,0.196395,0.140189,F=3-0_S=10_E=4000-100_B=0.3-False-200_R=0_T=tr...
10,11 / 0,100,0.3 / False / 200,31.274787,31.771646,285.943142,2.240907e+06,0.178796,0.032246,0.179571,0.138342,0.042926,0.207185,0.141440,F=11-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=...
8,0 / 0,100,1.0 / True / 200,31.345023,31.869590,256.057124,2.175368e+06,0.117883,0.027616,0.166182,0.137502,0.045397,0.213065,0.161515,F=0-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=tr...
11,11 / 0,100,1.0 / False / 200,31.777180,32.178982,199.905872,2.008987e+06,0.119231,0.028678,0.169346,0.144347,0.052112,0.228280,0.179190,F=11-0_S=100_E=4000-100_B=1.0-False-200_R=0_T=...
6,0 / 0,100,0.3 / False / 200,31.956958,32.283994,335.902873,2.562516e+06,0.159104,0.033565,0.183208,0.136483,0.054875,0.234254,0.166603,F=0-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=t...
1,/ 1 / 0,100,0.3 / False / 200,32.480483,32.784082,357.390505,2.681172e+06,0.166137,0.035914,0.189509,0.152318,0.055505,0.235595,0.169015,F=-1-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=...


,flow / init,n_samples,Beta / Anneal / Iters,1-Wasserstein,2-Wasserstein,Linear_MMD,Poly_MMD,RBF_MMD,Mean_MSE,Mean_L2,Mean_L1,Median_MSE,Median_L2,Median_L1,file
10,11 / 0,100,0.3 / False / 200,10.756324,10.983309,37.319660,67857.748868,0.277966,0.018514,0.136065,0.089979,0.026403,0.162491,0.095301,F=11-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=...
19,31 / 0,10,0.3 / False / 200,10.867163,11.054175,38.509407,70842.294130,0.212274,0.014497,0.120403,0.084808,0.023022,0.151729,0.093928,F=31-0_S=10_E=4000-100_B=0.3-False-200_R=0_T=t...
6,0 / 0,100,0.3 / False / 200,10.893063,11.066211,39.865068,71626.340571,0.267840,0.018125,0.134629,0.076955,0.026818,0.163762,0.093351,F=0-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=t...
0,/ 1 / 0,100,0.1 / False / 200,11.208975,11.353847,54.329199,104127.648665,0.386739,0.022971,0.151561,0.109317,0.025615,0.160048,0.099336,F=-1-0_S=100_E=4000-100_B=0.1-False-200_R=0_T=...
17,3 / 0,10,0.3 / False / 200,11.452064,11.643697,45.391412,83592.254740,0.250543,0.018569,0.136269,0.093056,0.029744,0.172465,0.105191,F=3-0_S=10_E=4000-100_B=0.3-False-200_R=0_T=tr...
8,0 / 0,100,1.0 / True / 200,11.622289,11.865332,25.645595,57227.425536,0.191970,0.014123,0.118841,0.094794,0.019760,0.140570,0.094688,F=0-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=tr...
3,/ 1 / 0,100,1.0 / True / 200,11.860675,12.060491,24.929401,44963.933736,0.197248,0.014387,0.119944,0.097341,0.019485,0.139588,0.095723,F=-1-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=t...
13,2 / 0,100,1.0 / True / 200,11.829410,12.166074,39.616708,71801.726848,0.221000,0.017887,0.133744,0.105964,0.022488,0.149959,0.108832,F=2-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=tr...
7,0 / 0,100,1.0 / False / 200,11.991930,12.197436,31.629680,53779.216275,0.211189,0.015579,0.124814,0.097280,0.016320,0.127748,0.094032,F=0-0_S=100_E=4000-100_B=1.0-False-200_R=0_T=t...
1,/ 1 / 0,100,0.3 / False / 200,12.099883,12.265173,62.921605,106805.622900,0.341624,0.026077,0.161483,0.122426,0.033955,0.184269,0.125988,F=-1-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=...


,flow / init,n_samples,Beta / Anneal / Iters,1-Wasserstein,2-Wasserstein,Linear_MMD,Poly_MMD,RBF_MMD,Mean_MSE,Mean_L2,Mean_L1,Median_MSE,Median_L2,Median_L1,file
6,0 / 0,100,0.3 / False / 200,2.435808,2.501811,1.693246,636.127148,0.149010,0.003142,0.056056,0.043104,0.004748,0.068908,0.052471,F=0-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=t...
5,0 / 0,100,0.1 / False / 200,2.539522,2.555976,4.049105,1819.760813,0.307905,0.009913,0.099566,0.080945,0.010477,0.102355,0.081498,F=0-0_S=100_E=4000-100_B=0.1-False-200_R=0_T=t...
10,11 / 0,100,0.3 / False / 200,2.499164,2.588631,1.587511,865.397515,0.153579,0.004350,0.065958,0.049778,0.005593,0.074788,0.055858,F=11-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=...
0,/ 1 / 0,100,0.1 / False / 200,2.589958,2.610542,3.874499,1545.657705,0.249935,0.009790,0.098944,0.079507,0.010688,0.103385,0.081121,F=-1-0_S=100_E=4000-100_B=0.1-False-200_R=0_T=...
1,/ 1 / 0,100,0.3 / False / 200,2.808396,2.927665,2.405302,811.316962,0.169372,0.006056,0.077817,0.057469,0.006652,0.081562,0.057728,F=-1-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=...
19,31 / 0,10,0.3 / False / 200,2.940971,3.053692,2.128839,831.018463,0.162983,0.004726,0.068742,0.046528,0.006566,0.081031,0.057806,F=31-0_S=10_E=4000-100_B=0.3-False-200_R=0_T=t...
17,3 / 0,10,0.3 / False / 200,3.184977,3.264391,3.496798,1603.930574,0.179627,0.007525,0.086749,0.053575,0.010515,0.102545,0.067148,F=3-0_S=10_E=4000-100_B=0.3-False-200_R=0_T=tr...
13,2 / 0,100,1.0 / True / 200,3.496464,3.680307,5.270275,2063.387291,0.193064,0.011683,0.108087,0.095955,0.008794,0.093778,0.076873,F=2-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=tr...
8,0 / 0,100,1.0 / True / 200,3.663607,3.824091,2.491143,1040.564402,0.176742,0.005726,0.075672,0.062465,0.005607,0.074882,0.049906,F=0-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=tr...
11,11 / 0,100,1.0 / False / 200,3.869577,4.019695,2.317310,965.353329,0.188960,0.008663,0.093077,0.075723,0.005360,0.073211,0.051778,F=11-0_S=100_E=4000-100_B=1.0-False-200_R=0_T=...


In [12]:
m = max(Counter(selected).values())
print("Max in top results:", m)
for k, c in Counter(selected).items():
    if c<m:
        continue
    print(k.replace(".csv", ".pdf"))

Max in top results: 3
F=2-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=trained-nn.pdf
F=31-0_S=10_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.pdf
F=3-0_S=10_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.pdf
F=11-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.pdf
F=0-0_S=100_E=4000-100_B=1.0-True-200_R=0_T=trained-nn.pdf
F=0-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.pdf
F=-1-0_S=100_E=4000-100_B=0.3-False-200_R=0_T=trained-nn.pdf
